# 层和块

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[-0.0162, -0.0076,  0.0441, -0.0361,  0.0645, -0.1857, -0.1766, -0.2695,
          0.1781,  0.2223],
        [-0.0076,  0.0196,  0.1254,  0.0010,  0.0323, -0.1256, -0.2768, -0.2145,
          0.1778,  0.1987]], grad_fn=<AddmmBackward0>)

## 自定义块

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层
    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[ 0.2153,  0.2559, -0.0703, -0.1197, -0.2836,  0.0018,  0.1618,  0.0868,
         -0.0895,  0.0595],
        [ 0.2968,  0.2494, -0.0119, -0.0958, -0.2515,  0.0594,  0.1177,  0.0719,
          0.0041,  0.0777]], grad_fn=<AddmmBackward0>)

## 顺序块

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0214,  0.2071, -0.0212,  0.1821,  0.2371,  0.1505,  0.2252,  0.1710,
          0.1667, -0.0456],
        [-0.0672,  0.3019, -0.0894,  0.1121,  0.2402,  0.0846,  0.2011,  0.0882,
          0.0703,  0.1019]], grad_fn=<AddmmBackward0>)

## 在前向传播函数中执行代码

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(0.1519, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.4053, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.3776, grad_fn=<SumBackward0>)

# 参数管理

In [10]:
import torch
from torch import nn
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.1687],
        [0.2601]], grad_fn=<AddmmBackward0>)

## 参数访问

In [11]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.1620,  0.3363,  0.0900, -0.0700, -0.2397, -0.2390,  0.1334,  0.1283]])), ('bias', tensor([0.2195]))])


### 目标参数

In [12]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2195], requires_grad=True)
tensor([0.2195])


In [13]:
net[2].weight.grad == None

True

### 一次性访问所有参数

In [14]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [15]:
net.state_dict()['2.bias'].data

tensor([0.2195])

### 从嵌套块收集参数

In [16]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3442],
        [-0.3441]], grad_fn=<AddmmBackward0>)

In [17]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [18]:
rgnet[0][1][0].bias.data

tensor([ 0.0595,  0.0005, -0.0896,  0.1575, -0.0217,  0.3796, -0.0028,  0.2393])

## 参数初始化

### 内置初始化

In [19]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0085,  0.0074,  0.0050,  0.0166]), tensor(0.))

In [20]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [21]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.5165, -0.4188, -0.1851, -0.5948])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化

In [22]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-5.6310,  9.9922, -0.0000, -7.4007],
        [ 7.7976,  5.2626,  6.7144, -0.0000]], grad_fn=<SliceBackward0>)

### 参数绑定

In [23]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),shared, nn.ReLU(),shared, nn.ReLU(),nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 延后初始化

# 自定义层

## 不带参数的层

In [24]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X - X.mean()

In [25]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [26]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [27]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-2.3283e-09, grad_fn=<MeanBackward0>)

## 带参数的层

In [28]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [29]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.5555, -0.8024,  0.5914],
        [-1.1150,  0.0025, -1.3849],
        [ 0.7315,  0.4526,  0.3786],
        [ 0.6035, -0.0193,  0.1129],
        [ 0.9838,  0.1276, -0.0907]], requires_grad=True)

In [30]:
linear(torch.rand(2, 5))

tensor([[1.7381, 1.2089, 0.8118],
        [2.6975, 1.2627, 1.2012]])

In [31]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[ 9.3849],
        [14.2803]])

# 读写文件

## 加载和保存张量

In [32]:
import torch
from torch import nn
from torch.nn import functional as F
x = torch.arange(4)
torch.save(x, 'x-file')

In [33]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [34]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [35]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 加载和保存模型参数

In [36]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [37]:
torch.save(net.state_dict(), 'mlp.params')

In [38]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [39]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

# GPU

In [40]:
!nvidia-smi

Fri May 17 10:11:42 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 Ti   WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   51C    P3             30W /  285W |    2729MiB /  12282MiB |     38%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 计算设备

In [41]:
import torch
from torch import nn
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [42]:
torch.cuda.device_count()

1

In [43]:
def try_gpu(i=0): #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')
def try_all_gpus(): #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]
try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## 张量与gpu

In [44]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

## 存储在GPU上

In [45]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [47]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.7685, 0.0200, 0.0523],
        [0.9700, 0.9466, 0.5373]], device='cuda:0')

## 复制

In [49]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [50]:
Y + Z

tensor([[1.7685, 1.0200, 1.0523],
        [1.9700, 1.9466, 1.5373]], device='cuda:0')

In [52]:
Z.cuda(0) is Z

True

## 神经网络与GPU

In [53]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [54]:
net(X)

tensor([[0.7412],
        [0.7412]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [55]:
net[0].weight.data.device

device(type='cuda', index=0)